The landsat_hires netcdf file can be found at http://glasslab.engr.ccny.cuny.edu/u/hannah/eas213/landsat_hires.nc

If you have the file on your computer, set the path to the location on disk:

```python
datapath = "/path/to/landsat_hires.nc"
```
if you are running the file via mybinder (the interactive version), then set the path to the managed version of the data:

```python
from quilt.data.story645.eas213 import landsat_hires
datapath = landsat_hires._data()
```

You can also install quilt and the data package on your computer to access the data. To do so, enter the following commands in a terminal or anaconda command prompt:

```bash
pip install quilt
quilt install story645/eas213
```


In [ ]:
#set datapath variable in this cell
datapath = '../data/landsat_hires.nc'

In [ ]:
import xarray as xr
ds = xr.open_dataset(datapath)

In [ ]:
import landsat_sample_functions as lf
# pull out the thermal radiances from from ds and convert them to temperature
tempK = lf.BT(ds['Thermal Radiances'], 10.9, 1)
#convert the temperature to Fahrenheit
tempF = 9/5*(tempK - 273) + 32

# How does temperature change over time?

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [ ]:
# print the min and max to get rough values for colormap
tempF.min(), tempF.max()

In [ ]:
# lets set the parameters
vmin = 20
vmax = 80
cmap = 'RdYlBu_r'

In [ ]:
# use the pandas library to clean up the dates
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html
import pandas as pd
datestr = pd.to_datetime(ds.time.values).strftime("%Y-%m-%d").values
num_frames = len(datestr)
datestr

In [ ]:
%%capture 
# %% capture supresses cell output, used here so that only the animation shows

# create figute
fig, ax = plt.subplots()
# plot the first tempF frame, 
# tell imshow the image is upside down
# use vmin, vmax, and cmap to make all images consistent
pcm = ax.imshow(tempF[0], vmin=vmin, vmax= vmax, cmap=cmap, 
                interpolation='none', origin='lower')
#create one colorbar that'll be used for all the images
cb = fig.colorbar(pcm)
#set the colorbar label, the xlabel, and the ylabel
cb.set_label(r"Temperature ($\degree F$)")
ax.set_title(datestr[0])
# animation function.  This is called sequentially
def animate(i):
    # we change the data being drawn
    # and use the tempF at index (time) i
    ax.set_title(datestr[i])
    pcm.set_data(tempF[i].values)
    return [pcm]


In [ ]:
ani = animation.FuncAnimation(fig, animate, frames=range(1,num_frames))
HTML(ani.to_jshtml())

In [ ]:
ani.save('temperature.html', dpi=80)

# Make a video for NDVI